In [35]:
import os
from os import listdir

In [135]:
with open("/Users/nerdai/Projects/migration/llama_index/llamaindex_registry.txt", "r") as f:
    packages = f.read().split('\n')

packages[:5]

['/Users/nerdai/Projects/migration/llama_index/llama-index-experimental',
 '/Users/nerdai/Projects/migration/llama_index/llama-index-legacy',
 '/Users/nerdai/Projects/migration/llama_index',
 '/Users/nerdai/Projects/migration/llama_index/llama-index-packs/llama-index-packs-retry-engine-weaviate',
 '/Users/nerdai/Projects/migration/llama_index/llama-index-packs/llama-index-packs-llava-completion']

### Attributes

1. Package name
2. Package version
3. Package kind (parent module)
4. Main classes
5. pypi link
6. github link

In [4]:
import toml

In [69]:
package = '/Users/nerdai/Projects/migration/llama_index/llama-index-integrations/llms/llama-index-llms-openai'
toml_path = os.path.join(package, "pyproject.toml")
pyproject = toml.load(toml_path)

In [70]:
pyproject['tool']['poetry']

{'authors': ['Your Name <you@example.com>'],
 'description': 'llama-index llms openai integration',
 'license': 'MIT',
 'name': 'llama-index-llms-openai',
 'packages': [{'include': 'llama_index/'}],
 'readme': 'README.md',
 'version': '0.0.1',
 'dependencies': {'python': '>=3.8.1,<3.12', 'llama-index-core': '^0.9.32'},
 'group': {'dev': {'dependencies': {'black': {'extras': ['jupyter'],
     'version': '<=23.9.1,>=23.7.0'},
    'codespell': {'extras': ['toml'], 'version': '>=v2.2.6'},
    'ipython': '8.10.0',
    'jupyter': '^1.0.0',
    'mypy': '0.991',
    'pre-commit': '3.2.0',
    'pylint': '2.15.10',
    'pytest': '7.2.1',
    'pytest-mock': '3.11.1',
    'ruff': '0.0.292',
    'tree-sitter-languages': '^1.8.0',
    'types-Deprecated': '>=0.1.0',
    'types-PyYAML': '^6.0.12.12',
    'types-protobuf': '^4.24.0.4',
    'types-redis': '4.5.5.0',
    'types-requests': '2.28.11.8',
    'types-setuptools': '67.1.0.0'}}}}

In [98]:
walker = os.walk(os.path.join(package, "llama_index"))

In [99]:
found_init = False
namespaces = ["llama_index"]

while not found_init:
    _, dirs, files = next(walker)
    namespaces += dirs
    if "__init__.py" in files:
        found_init = True

#### open __init__.py

In [111]:
namespace = os.path.join(*namespaces)
with open(os.path.join(package, *[namespace, "__init__.py"]), "r") as f:
    init = f.read()
all = eval(init.split("__all__")[-1].replace("=",""))
all

['OpenAI', 'Tokenizer', 'SyncOpenAI', 'AsyncOpenAI']

In [129]:
name = pyproject['tool']['poetry']['name']
version = pyproject['tool']['poetry']['version']
parent_module = os.path.join(*namespaces[:-1]).replace("/", ".")
pypi = f"https://pypi.org/project/{name}/"
classes = " ◦ ".join(all)
github_path = package.split("llama_index/")[-1]
github = f"https://github.com/nerdai/llama_index/tree/nerdai/migration-v0_10_0/{github_path}"

(name, version, parent_module, classes, pypi, github)

('llama-index-llms-openai',
 '0.0.1',
 'llama_index.llms',
 'OpenAI ◦ Tokenizer ◦ SyncOpenAI ◦ AsyncOpenAI',
 'https://pypi.org/project/llama-index-llms-openai/',
 'https://github.com/nerdai/llama_index/tree/nerdai/migration-v0_10_0/llama-index-integrations/llms/llama-index-llms-openai')

### Loop And Write CSV

In [161]:
def _get_namespaces(package):
    walker = os.walk(os.path.join(package, "llama_index"))
    namespaces = ["llama_index"]
    
    found_init = False
    while not found_init:
        _, dirs, files = next(walker)
        if "__init__.py" in files:
            found_init = True
        else:
            namespaces += dirs
            
    return namespaces

In [170]:
def _get_all_imports(package, namespaces):
    namespace = os.path.join(*namespaces)
    with open(os.path.join(package, *[namespace, "__init__.py"]), "r") as f:
        init = f.read()
    if "__all__" in init:
        return eval(init.split("__all__")[-1].replace("=",""))
    else:
        return []

In [174]:
def _init_csv():
    with open("llamaindex_packages.csv", "w") as f:
        f.write(f"name,version,parent_module,classes,pypi,github\n")

In [191]:
_init_csv()
for package in packages:
    print(package)
    if package.endswith("llama_index") or package.endswith("llama-index-core") or package.endswith("llama-index-legacy") or not package:
        continue

    toml_path = os.path.join(package, "pyproject.toml")
    pyproject = toml.load(toml_path)
    name = pyproject['tool']['poetry']['name']
    version = pyproject['tool']['poetry']['version']

    namespaces = _get_namespaces(package)
    parent_module = os.path.join(*namespaces[:-1]).replace("/", ".")
    pypi = f"https://pypi.org/project/{name}/"

    all_imports = _get_all_imports(package, namespaces)
    classes = " ◦ ".join(f"{el}" for el in all_imports)
    github_path = package.split("llama_index/")[-1]
    github = f"https://github.com/nerdai/llama_index/tree/nerdai/migration-v0_10_0/{github_path}"

    with open("llamaindex_packages.csv", "a") as f:
        f.write(f"{name},{version},{parent_module},{classes},{pypi},{github}\n")

/Users/nerdai/Projects/migration/llama_index/llama-index-experimental
/Users/nerdai/Projects/migration/llama_index/llama-index-legacy
/Users/nerdai/Projects/migration/llama_index
/Users/nerdai/Projects/migration/llama_index/llama-index-packs/llama-index-packs-retry-engine-weaviate
/Users/nerdai/Projects/migration/llama_index/llama-index-packs/llama-index-packs-llava-completion
/Users/nerdai/Projects/migration/llama_index/llama-index-packs/llama-index-packs-zephyr-query-engine
/Users/nerdai/Projects/migration/llama_index/llama-index-packs/llama-index-packs-timescale-vector-autoretrieval
/Users/nerdai/Projects/migration/llama_index/llama-index-packs/llama-index-packs-multi-document-agents
/Users/nerdai/Projects/migration/llama_index/llama-index-packs/llama-index-packs-multidoc-autoretrieval
/Users/nerdai/Projects/migration/llama_index/llama-index-packs/llama-index-packs-gmail-openai-agent
/Users/nerdai/Projects/migration/llama_index/llama-index-packs/llama-index-packs-neo4j-query-engine


In [183]:
package

''

In [182]:
toml_path

'pyproject.toml'

In [178]:
init.split("__all__")[-1].replace("=","")

'_  ["ApifyActor", "ApifyDataset"]\n'

In [164]:
package = "/Users/nerdai/Projects/migration/llama_index/llama-index-packs/llama-index-packs-tables"
_get_namespaces(package)

['llama_index', 'packs', 'tables']

In [163]:
walker = os.walk(os.path.join(package, "llama_index"))
namespaces = ["llama_index"]

In [160]:
namespaces

['llama_index']

In [150]:
next(walker)

('/Users/nerdai/Projects/migration/llama_index/llama-index-packs/llama-index-packs-tables/llama_index',
 ['packs'],
 [])

In [151]:
next(walker)

('/Users/nerdai/Projects/migration/llama_index/llama-index-packs/llama-index-packs-tables/llama_index/packs',
 ['tables'],
 [])

In [152]:
next(walker)

('/Users/nerdai/Projects/migration/llama_index/llama-index-packs/llama-index-packs-tables/llama_index/packs/tables',
 ['mix_self_consistency', 'chain_of_table'],
 ['__init__.py'])